<a href="https://colab.research.google.com/github/ant-nik/intelisys/blob/main/intelisys_pract_4_db_rules.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Использование базы данных в качестве машины логического вывода

Основные идеи:
- хранение сущностей в таблицах (рабочая память)
- индексы как альтернатива графов в RETE для быстрого доступа
- денормализация таблиц ч/з VIEW для сопоставления схожих сущностей
- производные факты как функции (параметризованные таблицы) или новые таблицы
- условия правил как SELECT
- продукции правил как INSERT
- как помечать обработанные факты? => любое правило это SELECT-UPDATE, помечающее факты как отработанные (???)
- как организовать цикл вставка фактов/логический вывод?
- как работать при отсутствии хранимых процедур? 
- как сделать итерационную обработку продукций до обработки всех фактов?


In [2]:
!pip install --quiet duckdb
!pip install --quiet jupysql 
!pip install --quiet duckdb-engine
!pip install --quiet pandas
!pip install --quiet matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.2/15.2 MB 72.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.7 MB/s eta 0:00:00


In [3]:
import duckdb
import pandas as pd

In [7]:
%sql SELECT 1, 2, 4

,1,2,4
0,1,2,4


In [2]:
import numpy
import pandas

Попытка аллокации 8Гб оперативной памяти

In [ ]:
data = numpy.random.random(500000000)
x = pandas.DataFrame(index=[], columns=data, copy=False)

In [3]:
import sys
sys.getsizeof(x)

8000000112

Запрос на поиск чисел

In [6]:
%sql select random() 

,random()
0,0.486827


In [3]:
%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
%sql duckdb:///test.db:
%sql SET memory_limit='4GB'

The sql extension is already loaded. To reload it, use:
  %reload_ext sql
 * duckdb:///test.db:


""


In [2]:
%sql CREATE TABLE test (value DOUBLE)

""


In [3]:
%%sql with recursive gen(i, x) as (
    select 1 as i, random() as x
    union all
    select i + 1, random() as x 
    from gen
    where i < 100000

)
INSERT INTO test select x from gen

FloatProgress(value=0.0, layout=Layout(width='100%'), style=ProgressStyle(bar_color='black'))

,Count
0,100000


In [3]:
%sql INSERT INTO test select value + random() from test limit 3276800000

FloatProgress(value=0.0, layout=Layout(width='100%'), style=ProgressStyle(bar_color='black'))

,Count
0,1638400000


In [3]:
%sql select count(*) from test

FloatProgress(value=0.0, layout=Layout(width='100%'), style=ProgressStyle(bar_color='black'))

,count_star()
0,3276800000


In [ ]:
%%sql select count(*) from test a cross join test b
where a.value > b.value


FloatProgress(value=0.0, layout=Layout(width='100%'), style=ProgressStyle(bar_color='black'))

In [5]:
%%sql 
EXPLAIN ANALYSE with cte as (
    select value, 
        lag(value) over wnd as prev,
        lead(value) over wnd as next
    from test
    WINDOW wnd as (ORDER BY value)
)
select sum(0.25*prev + 0.5*value + 0.25*next)
from cte

FloatProgress(value=0.0, layout=Layout(width='100%'), style=ProgressStyle(bar_color='black'))

(duckdb.IOException) IO Error: Could not write all bytes to file "/content/test.db:.tmp/duckdb_temp_storage-0.tmp": wanted=262144 wrote=147456
[SQL: EXPLAIN ANALYSE with cte as (
    select value, 
        lag(value) over wnd as prev,
        lead(value) over wnd as next
    from test
    WINDOW wnd as (ORDER BY value)
)
select sum(0.25*prev + 0.5*value + 0.25*next)
from cte]
(Background on this error at: https://sqlalche.me/e/14/e3q8)
